In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")

In [3]:
from ease_recommender import *
from npmi_recommender import *

import pickle as p

In [4]:
print("loading cache data...")
D = p.load(open("cached_data/movie_lens_preprocessed.p", "rb"))

row = D["userId"]
col = D["movieId"]
data = D["rating"]

movies = D["movies"]

print("done")

loading cache data...
done


In [5]:
def find_match_using_terms(terms, movies=movies, case_insensitive=False):
    assert type(terms) in (list, tuple, set)
    
    title = movies.title
    if case_insensitive:
        title = title.str.lower()
    
    matches = True
    for term in terms:
        matches &= title.str.contains(term)
        
    matches = np.where(matches)[0]

    if len(matches) > 1:
        raise Exception("Multiple matches found, filter down to a single match", movies.loc[matches, "title"].tolist())
        
    return matches[0]

In [6]:
mat = csr_matrix((data.astype(bool), (row, col))).astype(np.int64)

In [7]:
a = find_match_using_terms(["Sense and Sensibility", "1995"])
# a = find_match_using_terms(["Knives Out", "2019"])
# a = find_match_using_terms(["Witness", "1957"])

movies.loc[a]

title         Sense and Sensibility (1995)
genres                       Drama|Romance
imdbId                              114388
tmdbId                              4584.0
avg_rating                        7.891793
num_votes                            18677
Name: 16, dtype: object

In [8]:
b = find_match_using_terms(["Pride and Prejudice", "1995"])
# b = find_match_using_terms(["Death", "Nile", "1978"])
# b = find_match_using_terms(["Amadeus"])

movies.loc[b]

title         Pride and Prejudice (1995)
genres                     Drama|Romance
imdbId                            112130
tmdbId                          164721.0
avg_rating                       7.98947
num_votes                           2607
Name: 7382, dtype: object

In [9]:
lambda_ = optimize_lambda_using_a_to_b_matching_npmi(mat, a, b)

lambda_: 0.3819660112501051
error: 110.3819660112501
lambda_: 0.6180339887498948
error: 169.6180339887499
lambda_: 0.2360679774997897
error: 93.23606797749979
lambda_: 0.14077951786086368
error: 76.14077951786086
lambda_: 0.08700652695783664
error: 78.08700652695784
lambda_: 0.12640476530320013
error: 75.1264047653032
lambda_: 0.12057135932510735
error: 76.12057135932511
lambda_: 0.1306335117092338
error: 75.13063351170923
lambda_: 0.12848979019628198
error: 76.12848979019628
lambda_: 0.12640143009498114
error: 75.12640143009499
lambda_: 0.12417454121770662
error: 76.12417454121771
lambda_: 0.12555083423303137
error: 75.12555083423302
lambda_: 0.1250251370796564
error: 75.12502513707966
lambda_: 0.1247002383710816
error: 76.12470023837108
lambda_: 0.12522593552445657
error: 75.12522593552445
lambda_: 0.12490103681588177
error: 76.12490103681588
lambda_: 0.12510183526068194
error: 75.12510183526068
lambda_: 0.12497773499690731
error: 76.1249777349969
lambda_: 0.12505443317793286
error: 

In [10]:
a_to_b_error_metric_npmi_pop_weighted(mat, a, b, lambda_, lambda_penalty=False)

lambda_: 0.1250251370796564
error: 37.5


37.5

In [11]:
top_k = 20

In [12]:
# using normalized pointwise mutual information (popularity weighted)

similarity_scores = npmi_batch_popularity_weighted(mat, a, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
819,Emma (1996),Comedy|Drama|Romance,116191,3573.0,7.485110,6486
510,"Remains of the Day, The (1993)",Drama|Romance,107943,1245.0,7.783682,8651
492,Much Ado About Nothing (1993),Comedy|Romance,107616,11971.0,7.734885,11266
27,Persuasion (1995),Drama|Romance,114117,17015.0,8.076637,2719
352,Four Weddings and a Funeral (1994),Comedy|Romance,109831,712.0,7.284194,19987
35,Dead Man Walking (1995),Crime|Drama,112818,687.0,7.847888,17814
57,"Postman, The (Postino, Il) (1994)",Comedy|Drama|Romance,110877,11010.0,7.927828,10200
262,Like Water for Chocolate (Como agua para choco...,Drama|Fantasy|Romance,103994,18183.0,7.827167,9183
504,"Piano, The (1993)",Drama|Romance,107822,713.0,7.368306,12289


In [13]:
# using normalized pointwise mutual information (popularity weighted)

similarity_scores = npmi_batch_popularity_weighted(mat, b, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
10352,Pride & Prejudice (2005),Drama|Romance,414387,4348.0,7.703508,6645
14199,Persuasion (2007),Drama|Romance,844330,13949.0,7.719715,330
17597,North & South (2004),Drama|Romance,417349,147269.0,8.057732,400
16728,Northanger Abbey (2007),Drama|Romance,844794,18093.0,7.429658,201
13158,"Young Victoria, The (2009)",Drama|Romance,962736,18320.0,7.472767,679
11401,Becoming Jane (2007),Drama|Romance,416508,2977.0,7.129006,837
15923,Jane Eyre (2011),Drama|Romance,1229822,38684.0,7.451439,824
2984,Mansfield Park (1999),Comedy|Drama|Romance,178737,10399.0,7.523058,1191
15323,"King's Speech, The (2010)",Drama,1504320,45269.0,7.814929,17195


In [14]:
# average the two rankings

similarity_scores_a = npmi_batch_popularity_weighted(mat, a, lambda_)
similarity_ranking_a = np.full(len(similarity_scores_a), -1.0)
similarity_ranking_a[np.argsort(-similarity_scores_a)] = 1.0 - ((1.0 + np.arange(len(similarity_ranking_a)))/len(similarity_ranking_a))

similarity_scores_b = npmi_batch_popularity_weighted(mat, b, lambda_)
similarity_ranking_b = np.full(len(similarity_scores_b), -1.0)
similarity_ranking_b[np.argsort(-similarity_scores_b)] = 1.0 - ((1.0 + np.arange(len(similarity_ranking_b)))/len(similarity_ranking_b))

similarity_scores = similarity_ranking_a * similarity_ranking_b

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

for m in movies.loc[np.argsort(-similarity_scores)[:top_k], "title"]:
    print(m)

Persuasion (1995)
Emma (1996)
Mansfield Park (1999)
Roman Holiday (1953)
Pride & Prejudice (2005)
My Fair Lady (1964)
Sound of Music, The (1965)
Sabrina (1954)
Bridget Jones's Diary (2001)
Singin' in the Rain (1952)
Ever After: A Cinderella Story (1998)
Importance of Being Earnest, The (2002)
Persuasion (2007)
Room with a View, A (1986)
Breakfast at Tiffany's (1961)
Becoming Jane (2007)
Charade (1963)
Notting Hill (1999)
Gosford Park (2001)
Mary Poppins (1964)
